<font size="6">**Riemann-problem for acoustics**</font>
$$
\partial_t p = \text{div} \mathbf{v} \\ 
\partial_t \mathbf{v} = \nabla p 
$$

$$
u_0(x) 
= \begin{pmatrix} p_0(x) \\ \mathbf{v_0}(x)  \end{pmatrix} 
= \begin{cases}
\begin{pmatrix} 1.1 \\ \mathbf{0}  \end{pmatrix}, \quad x \in \Omega_L =(-0.5,0\phantom{.5}), \\
\begin{pmatrix} 0.9 \\ \mathbf{0}  \end{pmatrix}, \quad x \in  \Omega_R =(\phantom{-}0\phantom{.0},0.5),
\end{cases}
$$

In [ ]:
import os
import matplotlib.pyplot as plt
import numpy as np
import itertools as it
import re
import pickle
if not os.getcwd().endswith("spacetimeproject"):
    os.chdir("..")
    print(os.getcwd())
from mpp.python.mppy import mpp



def run(procs, confname, kwargs):
    log = mpp.run(procs, confname, kwargs, return_log=True)
    if not re.findall("GN_Error", log):
        return ""
    return log

def nn(ls):
    for l in ls:
        if l:
            yield l

In [ ]:
mpp.build()

In [ ]:
lower_first = lambda *ls: list(sorted(list(it.product(*ls)), key=lambda x: sum(x)))

In [ ]:
logs = []

In [ ]:
mpp.build()

In [ ]:
procs = {0:1,1:2,2:8,3:16,4:32,5:32,6:32,7:32}
for (sdeg, tdeg), l in it.product([(0,1),(1,2),(2,3),(3,3)],[1,2,3,4,5,6]):
    logs.append(run(procs[l], "riemann", {"level":l, "plevel":l, "space_deg":sdeg, "time_deg":tdeg}))
       

In [ ]:
ff = lambda regex, log: float(re.findall(regex, log)[0])
fi = lambda regex, log: int(re.findall(regex, log)[0])

In [ ]:


def log_to_data(logg):
    l2 = []
    h = []
    sdeg = []
    tdeg = []
    for log in nn(logg):
        l2.append(ff("L2_Error                (.*)\n", log))
        h.append(ff("Mesh width:     \[(.*),", log))
        sdeg.append(fi("space_deg............................... (\d)",log))
        tdeg.append(fi("time_deg................................ (\d)",log))
    data = list(zip(h, l2, zip(sdeg, tdeg)))
    class DataSet:
        def __init__(self, data):
            self.data = data
            self.degrees = []
            self.degree_data = {}
            for group, items in it.groupby(sorted(data, key=lambda x:x[2]), lambda x:x[2]):        
                self.degrees.append(group)
                self.degree_data[group]= list(items)
                
            
    ds = DataSet(data)
    return ds

def data_to_graph(data, s = 0):
    hs = np.array([2**(-i) for i in range(1,7)])
    hh = ["2^{-"+str(k)+"}" for k in range(1,7)]
    latex = r"\hline (p,q)\text{\\} h & " +  " & ".join(map(str,np.round(hs,4))) +r"\\ \hline" +  "\n"
    for group, items in [(g,data.degree_data[g]) for g in data.degrees]:
        
        #print(group)
        #for item in items:
        #    print("\t",item[:2])
        #print([i[:2] for i in items])
        h, err = list(zip(*[i[:2] for i in items])) 
        #print(temp)
        plt.loglog(h[s:], err[s:], label=str(group), marker="x")
        #print(h[1:])
        latex += str(group) +" & "+ " & ".join(map(str, np.round(err,4))) + r"\\ \hline" + "\n" 
    
    plt.plot(hs,hs**0.5/7, color="black", ls="-", label="$\mathcal{O}(\sqrt{h})$")
    plt.plot(hs[:4],hs[:4]/5, color="black",ls="dotted", label="$\mathcal{O}(h)$")
    plt.xticks(hs, np.round(hs,4))
    plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    return r"\begin{array}{|r|r|}" + latex + r"\end{array}"


In [ ]:
old_data = log_to_data(pickle.load(open("logs","rb")))
latex = data_to_graph(old_data, s=1)
display(Latex(latex))
print(latex)

In [ ]:
data = log_to_data(logs)
latex = data_to_graph(data)
display(Latex(latex))
print(latex)

In [ ]:
from IPython.display import display, Math, Latex


In [ ]:
data_to_graph(old_data, s=1)

In [ ]:
1/0.0625

$$
Bw_1 =
\begin{pmatrix}
0 & 1 & 1\\
1 & 0 & 0\\
1 & 0 & 0
\end{pmatrix}
\begin{pmatrix}
\frac{1}{2}\sqrt2\\
\frac{1}{2} \\
\frac{1}{2}
\end{pmatrix} = 
\begin{pmatrix}
1\\
\frac{1}{2}\sqrt2 \\
\frac{1}{2}\sqrt2
\end{pmatrix} = \sqrt2\begin{pmatrix}
\frac{1}{2}\sqrt2\\
\frac{1}{2}  \\
\frac{1}{2} 
\end{pmatrix}
= \lambda_1 w_1
$$

In [ ]:
h = np.array([1/2,1/4,1/8,1/16,1/32])
l = [0.11224, 0.07875, 0.05636, 0.04068, 0.02964]
#l8 = [1.12238485e-01, 7.87537036e-02, 5.63575516e-02, 4.06773403e-02, 2.96380844e-02]

plt.loglog(h,l, marker = "x")

#plt.loglog(h,l8, marker = "x", ls = "--")
plt.loglog(h,h**0.5/6)
plt.xticks(h,h)

In [ ]:
help(plt.plot)